In [15]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler
import pickle

In [16]:
current_year = 2023

In [17]:
market = Market()
fin = ADatabase("financial")
umod = UniversalModeler()
sec = SEC()

In [18]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [19]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity']

In [20]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [21]:
included_columns.extend(factors)

In [22]:
training_sets = []

In [23]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        filing = filing.groupby(["year","quarter"]).mean().reset_index()
        ticker_data = prices.copy()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["adjclose"].shift(-4)
        ticker_data = ticker_data.merge(filing,on=["year","quarter"],how="left").reset_index()
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  1%|██                                                                                                                                                                         | 6/490 [00:01<01:59,  4.06it/s]

"['entitycommonstocksharesoutstanding'] not in index"


  2%|██▊                                                                                                                                                                        | 8/490 [00:02<02:22,  3.38it/s]

"['stockholdersequity'] not in index"


  4%|██████▌                                                                                                                                                                   | 19/490 [00:06<02:45,  2.84it/s]

...

 97%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 474/490 [02:52<00:08,  1.99it/s]

"['propertyplantandequipmentnet'] not in index"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 489/490 [02:58<00:00,  1.89it/s]

"['cashandcashequivalentsatcarryingvalue'] not in index"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [02:58<00:00,  2.74it/s]


In [24]:
data = pd.concat(training_sets)

In [25]:
data = data.dropna()

In [26]:
fin.connect()
for modeler in [umod]:
    for year in tqdm(range(current_year,current_year+1)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - 4)].reset_index(drop=True)
        set_name = "models"
        stuff = modeler.recommend_model(training_slice,factors,tf=False)
fin.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.96s/it]


In [27]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

In [28]:
fin.connect()
models = fin.store("models",stuff)
fin.disconnect()